In [ ]:
#!pip install python-telegram-bot

In [ ]:
import telegram
from binance.client import Client

In [ ]:
API_KEY = os.getenv('BINANCE_API_KEY')
API_SECRET = os.getenv('BINANCE_API_SECRET')
TELEGRAM_API_TOKEN = os.getenv('TELEGRAM_API_TOKEN')

bot = telegram.Bot(TELEGRAM_API_TOKEN)
client = Client(API_KEY,API_SECRET)

In [ ]:
print(bot.get_me())

In [ ]:
from telegram.ext import Updater
updater = Updater(token=TELEGRAM_API_TOKEN, use_context=True)

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [ ]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

In [ ]:
from telegram.ext import CommandHandler
dispatcher = updater.dispatcher
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [ ]:
from datetime import datetime as dt
dt.now()

In [ ]:
client.futures_create_order(symbol="ETHUSDT", side="SELL",type="LIMIT",timeInForce="GTC",quantity=0.01,price=326,timestamp=dt.now())

In [ ]:
def echo(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

from telegram.ext import MessageHandler, Filters
echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

In [ ]:
j = updater.job_queue

In [ ]:
LIMITS = dict({'TRXUSDT':[0.01890,0.01920]})
TEST = False

In [ ]:
def callback_minute(context: telegram.ext.CallbackContext):
    global LIMITS, TEST
    pos = list(filter(lambda f:(f['symbol'] in list(LIMITS.keys())),client.futures_account()['positions']))
    if pos:
        for position in pos:
            text = ""
            symbol = position['symbol']
            mark_price = client.futures_mark_price(symbol=symbol)['markPrice']
            text += (str(symbol) + " Price : " + str(mark_price))
            text += ".\n"
            text += ("Unrealized Profit : "+ position['unrealizedProfit'])
            text += ".\n"
            text += ("Entry Price : "+ position['entryPrice'])
            if (float(mark_price) > LIMITS[symbol][1]) or (float(mark_price) < LIMITS[symbol][0]):
                context.bot.send_message(chat_id='@jeevantest', text=text)
    if TEST:
        context.bot.send_message(chat_id='@jeevantest', text="Test message")

job_minute = j.run_repeating(callback_minute, interval=60, first=0)

In [ ]:
TEST = True

In [ ]:
def callback_minute(context: telegram.ext.CallbackContext):
    pos = list(filter(lambda f:(f['symbol']=='XRPUSDT'),client.futures_account()['positions']))
    if pos:
        text = ""
        position = pos[0]
        mark_price = client.futures_mark_price(symbol=position['symbol'])['markPrice']
        text += (str(position['symbol']) + " Price : " + str(mark_price))
        text += ".\n"
        text += ("Unrealized Profit : "+ position['unrealizedProfit'])
        text += ".\n"
        text += ("Entry Price : "+ position['entryPrice'])
        if (float(mark_price) > 0.21) or (float(mark_price) < 0.2050):
            context.bot.send_message(chat_id='@jeevantest', text=text)

job_minute = j.run_repeating(callback_minute, interval=120, first=0)

In [ ]:
updater.start_polling()

In [ ]:
from telegram import InlineQueryResultArticle, InputTextMessageContent, InputMediaPhoto
def inline_caps(update, context):
    query = update.inline_query.query
    if not query:
        return
    results = list()
    results.append(
        InlineQueryResultArticle(
            id=query.upper(),
            title='Caps3',
            input_message_content= InputMediaPhoto("https://images.indianexpress.com/2020/07/TEEJ-1-1.jpg")
        )
    )
    context.bot.answer_inline_query(update.inline_query.id, results)

from telegram.ext import InlineQueryHandler
inline_caps_handler3 = InlineQueryHandler(inline_caps)


In [ ]:
dispatcher.remove_handler(inline_caps_handler)

In [ ]:
j.jobs()

In [ ]:
for job in j.jobs():
    job.schedule_removal()

In [ ]:
j.jobs()